In [1]:
import Pkg
if "MKL" in keys(Pkg.project().dependencies)
  using MKL
end
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/Dev/Workspaces/ParallelLeastSquares/benchmark`


Status `~/Documents/Dev/Workspaces/ParallelLeastSquares/benchmark/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
  [8e7c35d0] BlockArrays v1.6.3
  [0a1fb500] BlockDiagonals v0.2.0
  [a93c6f00] DataFrames v1.7.0
  [31c24e10] Distributions v0.25.120
  [42fd0dbc] IterativeSolvers v0.9.4
  [9df87fff] MMDeweighting v1.0.0-DEV `https://github.com/qhengncsu/MMDeweighting.jl#main`
  [08abe8d2] PrettyTables v2.4.0
  [5d646e68] QBSolvers v0.1.0 `..`
  [6f49c342] RCall v0.14.8
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics v1.10.0


In [2]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 20 × Intel(R) Core(TM) i9-10900KF CPU @ 3.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 20 virtual cores)


In [3]:
using QBSolvers, LinearAlgebra, Distributions, Statistics

# OLS

In [4]:
n, p = 16*512, 2*512;
Σ = [(0.8)^abs(i-j) for i in 1:p, j in 1:p];
A = Transpose(rand(MvNormal(zeros(p), Σ), n)) |> Matrix;
x0 = 0.1*ones(p);
b = A*x0 + 1/sqrt(p)*randn(n);
x_init = zeros(p)
λ = 1e3
n_blk = p

1024

## QUB

In [5]:
x, r, stats = @time solve_OLS(A, b, x_init, n_blk; maxiter=10^2, use_qub=true, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS(A, b, x_init, n_blk; maxiter=10^4, use_qub=true, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            544ms /  38.2%           46.0MiB /  27.2%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
OLS Loop                   1    122ms   58.9%   122ms   2.38MiB   19.1%  2.38MiB
  update gₙ               99   1.95ms    0.9%  19.7μs     0.00B    0.0%    0.00B
  dₙ₊₁ = H⁻¹gₙ            99   95.0μs    0.0%   959ns     0.00B    0.0%    0.00B
  convergence check      100   35.8μs    0.0%   358ns     0.00B    0.0%    0.00B
  update xₙ₊₁             99   19.7μs    0.0%   199ns     0.00B    0.0%    0.00B
init recurrences           1   61.7ms   29.7%  61.7ms   1.87MiB   15.0%  1.87MiB
  compute -∇₀              

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           35.4ms /  99.9%           8.15MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   19.9ms   56.4%  19.9ms   8.00MiB   98.3%  8.00MiB
OLS Loop                   1   11.8ms   33.4%  11.8ms   3.09KiB    0.0%  3.09KiB
  update gₙ              607   10.7ms   30.2%  17.6μs     0.00B    0.0%    0.00B
  dₙ₊₁ = H⁻¹gₙ           607    576μs    1.6%   949ns     0.00B    0.0%    0.00B
  convergence check      608    208μs    0.6%   341ns     0.00B    0.0%    0.00B
  update xₙ₊₁            607    112μs    0.3%   184ns     0.00B    0.0%    0.00B
init recurrences           

  6.779078 seconds (13.28 M allocations: 867.589 MiB, 4.57% gc time, 99.63% compilation time)
  0.035539 seconds (999 allocations: 8.226 MiB)


## L-BFGS, no preconditioner

In [6]:
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init, n_blk; maxiter=10^2, precond=:none, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init, n_blk; maxiter=10^4, precond=:none, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           36.9ms /  99.9%           8.27MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   30.2ms   81.9%  30.2ms   8.00MiB   96.8%  8.00MiB
OLS Loop                   1   4.99ms   13.5%  4.99ms   7.34KiB    0.1%  7.34KiB
  w = (AᵀA+λI)dₙ₊₁       100   2.29ms    6.2%  22.9μs     0.00B    0.0%    0.00B
  init gradient            1   1.87ms    5.1%  1.87ms     0.00B    0.0%    0.00B
  compute dₙ₊₁           100    541μs    1.5%  5.41μs     0.00B    0.0%    0.00B
  update L-BFGS cache    100   67.1μs    0.2%   671ns     0.00B    0.0%    0.00B
  backtracking lin...    10

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           44.3ms /  99.9%           8.27MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   29.6ms   66.9%  29.6ms   8.00MiB   96.8%  8.00MiB
OLS Loop                   1   12.7ms   28.7%  12.7ms   7.34KiB    0.1%  7.34KiB
  w = (AᵀA+λI)dₙ₊₁       385   7.39ms   16.7%  19.2μs     0.00B    0.0%    0.00B
  compute dₙ₊₁           385   2.37ms    5.3%  6.14μs     0.00B    0.0%    0.00B
  init gradient            1   1.78ms    4.0%  1.78ms     0.00B    0.0%    0.00B
  update L-BFGS cache    385    268μs    0.6%   695ns     0.00B    0.0%    0.00B
  backtracking lin...    38

  3.179023 seconds (1.98 M allocations: 133.164 MiB, 0.53% gc time, 98.83% compilation time)
  0.044501 seconds (938 allocations: 8.346 MiB)


In [7]:
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init, n_blk; maxiter=10^2, precond=:qub, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init, n_blk; maxiter=10^4, precond=:qub, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           25.9ms /  99.9%           8.31MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   19.8ms   76.2%  19.8ms   8.00MiB   96.3%  8.00MiB
OLS Loop                   1   4.26ms   16.4%  4.26ms   7.34KiB    0.1%  7.34KiB
  init gradient            1   1.73ms    6.7%  1.73ms     0.00B    0.0%    0.00B
  w = (AᵀA+λI)dₙ₊₁        93   1.72ms    6.6%  18.4μs     0.00B    0.0%    0.00B
  compute dₙ₊₁            93    552μs    2.1%  5.94μs     0.00B    0.0%    0.00B
  update L-BFGS cache     93   61.0μs    0.2%   656ns     0.00B    0.0%    0.00B
  backtracking lin...     9

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           24.6ms /  99.9%           8.31MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   18.6ms   75.4%  18.6ms   8.00MiB   96.3%  8.00MiB
OLS Loop                   1   4.30ms   17.5%  4.30ms   7.34KiB    0.1%  7.34KiB
  init gradient            1   1.78ms    7.2%  1.78ms     0.00B    0.0%    0.00B
  w = (AᵀA+λI)dₙ₊₁        93   1.71ms    7.0%  18.4μs     0.00B    0.0%    0.00B
  compute dₙ₊₁            93    552μs    2.2%  5.93μs     0.00B    0.0%    0.00B
  update L-BFGS cache     93   63.3μs    0.3%   680ns     0.00B    0.0%    0.00B
  check convergence       9

  0.026149 seconds (1.09 k allocations: 8.398 MiB)
  0.024826 seconds (1.09 k allocations: 8.398 MiB)


# QREG

In [8]:
h = QBSolvers.default_bandwidth(A)
q = 0.5
b = A*x0 + rand(TDist(1.5), n) .- Statistics.quantile(TDist(1.5), q);

## QUB

In [9]:
x, r, stats = @time solve_QREG(A, b, x_init, n_blk; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);
x, r, stats = @time solve_QREG(A, b, x_init, n_blk; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            950ms / 100.0%           35.0MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
QREG Loop                  1    899ms   94.6%   899ms   26.5MiB   75.8%  26.5MiB
  inner solve             28    336ms   35.4%  12.0ms   2.94KiB    0.0%     107B
    update gₙ          17.1k    305ms   32.1%  17.9μs     0.00B    0.0%    0.00B
    dₙ₊₁ = H⁻¹gₙ       17.1k   16.3ms    1.7%   955ns     0.00B    0.0%    0.00B
    convergence check  17.1k   5.63ms    0.6%   329ns     0.00B    0.0%    0.00B
    update xₙ₊₁        17.1k   3.32ms    0.3%   194ns     0.00B    0.0%    0.00B
  update rₙ = b - Axₙ     2

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            464ms / 100.0%           8.46MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
QREG Loop                  1    439ms   94.5%   439ms   11.9KiB    0.1%  11.9KiB
  inner solve             28    344ms   74.1%  12.3ms   2.94KiB    0.0%     107B
    update gₙ          17.1k    313ms   67.4%  18.3μs     0.00B    0.0%    0.00B
    dₙ₊₁ = H⁻¹gₙ       17.1k   16.5ms    3.6%   968ns     0.00B    0.0%    0.00B
    convergence check  17.1k   5.63ms    1.2%   329ns     0.00B    0.0%    0.00B
    update xₙ₊₁        17.1k   3.09ms    0.7%   181ns     0.00B    0.0%    0.00B
  update rₙ = b - Axₙ     2

  2.201763 seconds (3.47 M allocations: 239.988 MiB, 1.46% gc time, 79.17% compilation time)
  0.464430 seconds (1.92 k allocations: 8.606 MiB)


## L-BFGS

In [10]:
x, r, stats = @time solve_QREG_lbfgs(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);
x, r, stats = @time solve_QREG_lbfgs(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            588ms / 100.0%           22.3MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
QREG Loop                  1    552ms   94.0%   552ms   13.9MiB   62.2%  13.9MiB
  inner solve             28    104ms   17.7%  3.71ms   7.34KiB    0.0%     269B
    init gradient         28   46.0ms    7.8%  1.64ms     0.00B    0.0%    0.00B
    w = (AᵀA+λI)dₙ₊₁   1.97k   39.3ms    6.7%  20.0μs     0.00B    0.0%    0.00B
    compute dₙ₊₁       1.97k   12.7ms    2.2%  6.43μs     0.00B    0.0%    0.00B
    update L-BFGS ...  1.97k   1.38ms    0.2%   699ns     0.00B    0.0%    0.00B
    backtracking l...  1.97

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            176ms / 100.0%           8.46MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
QREG Loop                  1    151ms   85.9%   151ms   13.4KiB    0.2%  13.4KiB
  inner solve             28    103ms   58.4%  3.67ms   7.34KiB    0.1%     269B
    init gradient         28   44.8ms   25.5%  1.60ms     0.00B    0.0%    0.00B
    w = (AᵀA+λI)dₙ₊₁   1.97k   39.9ms   22.7%  20.2μs     0.00B    0.0%    0.00B
    compute dₙ₊₁       1.97k   12.2ms    6.9%  6.19μs     0.00B    0.0%    0.00B
    update L-BFGS ...  1.97k   1.33ms    0.8%   675ns     0.00B    0.0%    0.00B
    backtracking l...  1.97

  1.493979 seconds (1.89 M allocations: 133.026 MiB, 1.75% gc time, 87.37% compilation time)
  0.176127 seconds (2.02 k allocations: 8.628 MiB)
